In [1]:
!gpustat

dgx01                     Mon Sep 15 20:01:50 2025  535.183.06
[0] NVIDIA A100-SXM4-80GB | 32°C,  ?? % | 46539 / 81920 MB | taravichet(46526M)
[1] NVIDIA A100-SXM4-80GB | 28°C,  ?? % |    87 / 81920 MB |
[2] NVIDIA A100-SXM4-80GB | 28°C,  ?? % |    87 / 81920 MB |
[3] NVIDIA A100-SXM4-80GB | 29°C,  ?? % |    87 / 81920 MB |
[4] NVIDIA A100-SXM4-80GB | 33°C,  ?? % |     1 / 81920 MB |
[5] NVIDIA A100-SXM4-80GB | 30°C,  ?? % |     1 / 81920 MB |
[6] NVIDIA A100-SXM4-80GB | 32°C,  ?? % | 32459 / 81920 MB | sarmistha(32446M)
[7] NVIDIA A100-SXM4-80GB | 33°C,  ?? % | 70739 / 81920 MB | sarmistha(70726M)


In [2]:
# import os 
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [1]:
#can add different domains in this and increase the max document argument to get more news articles

import os
import wikipediaapi
import pandas as pd
import time
import zipfile
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Define Wikipedia API with User-Agent
wiki_wiki = wikipediaapi.Wikipedia(
    user_agent="FinancialRAG/1.0 (ishani@example.com)", language="en"
)

# Define broad finance-related Wikipedia categories
finance_categories = [
    "Finance",
    "Stock market",
    "Banking",
    "Investment",
    "Taxation",
    "Insurance",
    "Loans",
    "Trading",
    "Cryptocurrency",
    "General Finance",
    "Mortgage",
    "Money",
    "Personal finance",
    "Debit Cards",
    "Credit Cards",
    "Finance in India",
    "Stock market in India",
    "Banking in India",
    "Investment in India",
    "Taxation in India",
    "Insurance in India",
    "Loans in India",
    "Trading in India",
    "Cryptocurrency in India",
    "General Finance in India",
    "Mortgage in India",
    "Money in India",
    "Personal finance in India",
    "Debit Cards in India",
    "Credit Cards in India",
    "Financial Laws & Regulations",
    "Anti-Money Laundering (AML)",
    "Know Your Customer (KYC)",
    "Basel Accords",
    "Dodd-Frank Act",
    "SEBI Regulations (India)",
    "RBI Policies (India)",
    "Actuarial Science",
    "Reinsurance",
    "Life Insurance",
    "Health Insurance",
    "Property & Casualty Insurance",
    "Microinsurance",
    "Real Estate Investment Trusts (REITs)",
    "Commercial Real Estate",
    "Housing Markets",
    "Real Estate Valuation",
    "Property Taxation",
    "Healthcare Finance",
    "Energy Finance",
    "Agricultural Finance",
    "Infrastructure Finance",
    "Defense & Military Finance",
    "Decentralized Finance (DeFi)",
    "Blockchain Applications in Finance",
    "AI in Financial Services",
    "Algorithmic Trading",
    "High-Frequency Trading (HFT)",
    "Web3 & Finance",
    "Quantum Computing in Finance",
    "Monetary Policy",
    "Fiscal Policy",
    "Economic Indicators",
    "Inflation & Deflation",
    "Interest Rates",
    "GDP & Economic Growth",
    "Labor Market & Employment",
    "Recession & Economic Cycles",
    "Corporate Finance",
    "Mergers and Acquisitions",
    "Private Equity",
    "Venture Capital",
    "Financial Planning",
    "Risk Management",
    "Financial Modeling",
    "Business Valuation",
    "Wealth Management",
    "Capital Markets",
    "Asset Management",
    "Treasury Management",
    "Indian Economy",
    "Monetary Policy of India",
    "Fiscal Policy of India",
    "Reserve Bank of India (RBI) Policies",
    "Union Budget of India",
    "Economic Survey of India",
    "Make in India Initiative",
    "Startup India & MSME Financing",
    "Goods and Services Tax (GST) in India",
    "Inflation in India",
    "Public Sector Banks (PSBs) in India",
    "Private Sector Banks in India",
    "Regional Rural Banks (RRBs) in India",
    "Cooperative Banks in India",
    "Non-Banking Financial Companies (NBFCs) in India",
    "Microfinance Institutions (MFIs) in India",
    "Indian Payment Banks",
    "Unified Payments Interface (UPI) & Digital Payments in India",
    "Financial Inclusion in India (Jan Dhan Yojana, PM Mudra Yojana, etc.)",
    "National Stock Exchange (NSE) & Bombay Stock Exchange (BSE)",
    "Securities and Exchange Board of India (SEBI) Regulations",
    "Sensex & Nifty Indices",
    "Mutual Funds in India",
    "Exchange-Traded Funds (ETFs) in India",
    "Commodity Trading in India",
    "Derivative Markets in India",
    "Foreign Direct Investment (FDI) in India",
    "Foreign Institutional Investors (FII) in India",
    "Angel Investing & Venture Capital in India",
    "Initial Public Offerings (IPOs) in India",
    "Direct Taxes in India (Income Tax, Corporate Tax, etc.)",
    "Indirect Taxes in India (GST, Excise Duty, Customs Duty, etc.)",
    "Tax Exemptions & Deductions under Indian Tax Laws",
    "Provident Fund (EPF, PPF, NPS) in India",
    "National Pension System (NPS) in India",
    "Public Provident Fund (PPF) in India",
    "Employee Provident Fund (EPF) in India",
    "Atal Pension Yojana (APY)",
    "Pradhan Mantri Vaya Vandana Yojana (PMVVY)",
    "Sukanya Samriddhi Yojana (SSY)",
    "Agricultural Income & Rural Taxation in India",
    "Education Loans in India",
    "Home Loans in India",
    "Agriculture & Rural Credit in India",
    "Priority Sector Lending (PSL) in India",
    "Microfinance & Self-Help Groups (SHGs) in India",
    "Credit Rating Agencies in India (CRISIL, ICRA, CARE, etc.)",
    "Insurance Regulatory and Development Authority of India (IRDAI)",
    "Life Insurance in India (LIC, HDFC Life, etc.)",
    "Health Insurance in India (Ayushman Bharat, etc.)",
    "Motor & Vehicle Insurance in India",
    "Crop Insurance (Pradhan Mantri Fasal Bima Yojana - PMFBY)",
    "Reinsurance Market in India",
    

]

def get_category_articles(category_name, depth=5, max_articles=10000):
    """
    Recursively fetch articles from a Wikipedia category up to a given depth.
    """
    category = wiki_wiki.page(f"Category:{category_name}")
    articles = set()
    queue = [(category, 0)]
    
    print(f"\nFetching articles from: {category_name}")

    while queue:
        cat, level = queue.pop(0)
        if level > depth:
            continue

        for title, page in cat.categorymembers.items():
            if page.ns == wikipediaapi.Namespace.MAIN and title not in articles:
                articles.add(title)
                print(f"Fetched {len(articles)}: {title}")

            elif page.ns == wikipediaapi.Namespace.CATEGORY:
                queue.append((page, level + 1))

            if len(articles) >= max_articles:
                return articles  # Stop if max limit is reached

    return articles

# Collect articles from all finance-related categories
all_articles = set()
for category in finance_categories:
    articles = get_category_articles(category, depth=5, max_articles=5000)
    all_articles.update(articles)

print(f"\nTotal unique articles collected: {len(all_articles)}")

# Fetch article content
documents = []
for i, title in enumerate(tqdm(all_articles, desc="Fetching Wikipedia Articles"), start=1):
    page = wiki_wiki.page(title)
    if page.exists():
        documents.append((title, page.text))
    
    if i % 5 == 0:  # Avoid hitting Wikipedia rate limits
        time.sleep(1)

# Save raw data to CSV
csv_filename = "wikipedia_finance_articles.csv"
df_articles = pd.DataFrame(documents, columns=["Title", "Content"])
df_articles.to_csv(csv_filename, index=False)

# 🔹 ZIP the extracted articles
zip_filename = "wikipedia_finance_articles.zip"
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)
    print(f"✅ Saved documents to {zip_filename}")

# Remove the original CSV after zipping
os.remove(csv_filename)

print("📂 Zipping complete. Ready for reference!")

# Split text into chunks for RAG
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
split_documents = []
for title, text in documents:
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        split_documents.append((title, chunk))

# Use Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-Qwen2-1.5B-instruct")

# Create FAISS vector store
vectorstore = FAISS.from_texts([doc[1] for doc in split_documents], embedding_model)
vectorstore.save_local("./faiss_index")
print("🔹 Vector store created successfully!")

# Load user queries
df_queries = pd.read_csv("./queries.csv")

def retrieve_context(query, vectorstore, k=10, threshold=0.3):
    results = vectorstore.similarity_search_with_relevance_scores(query, k=k)
    filtered_results = [doc.page_content for doc, score in results if score >= threshold]
    return "\n\n---\n\n".join(filtered_results) if filtered_results else "No relevant context found."

# Retrieve and store context
df_queries["context"] = df_queries["user"].progress_apply(lambda query: retrieve_context(query, vectorstore))
df_queries.to_csv("./queries_with_context_latest.csv", index=False, encoding="utf-8", quoting=pd.io.common.csv.QUOTE_MINIMAL)
print("✅ Context saved successfully!")



Fetching articles from: Finance
Fetched 1: Finance
Fetched 2: Agricultural finance
Fetched 3: Approved Publication Arrangement
Fetched 4: Asset
Fetched 5: Austerity
Fetched 6: Bachelor of Finance
Fetched 7: Yvon Sana Bangui
Fetched 8: Capital Markets Union
Fetched 9: Capitalism
Fetched 10: CGAP (Thinktank)
Fetched 11: Climate finance in Zimbabwe
Fetched 12: Ettore Conti of Verampio, Count of Verampio
Fetched 13: Designated Professional Body
Fetched 14: DreamAhead College Investment Plan
Fetched 15: Exit (investing)
Fetched 16: Financial Modeling World Cup
Fetched 17: Financial stability
Fetched 18: Industrial and provident society
Fetched 19: Just Energy Transition Partnership
Fetched 20: Lynnette Khalfani-Cox
Fetched 21: Liquidation
Fetched 22: Look-through earnings
Fetched 23: Master of Applied Finance
Fetched 24: Master of Finance
Fetched 25: Microsoft Excel Collegiate Challenge
Fetched 26: Morningstar Rating for Funds
Fetched 27: Non-financial asset
Fetched 28: Online portfolio se

Fetching Wikipedia Articles:  16%|█▌        | 4438/27970 [1:12:58<6:03:12,  1.08it/s] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
#this code block will return the contexts
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from huggingface_hub import login
import csv

# Load environment variables
load_dotenv()

# Define paths
csv_knowledge_base_path = "./wikipedia_finance_articles.csv"  # Now using CSV instead of ZIP
csv_queries_path = "./user_queries.csv"
vectorstore_path = "./faiss_index"
output_csv_path = "./queries_with_context_latest.csv"

# Login to Hugging Face
login("<hf_api>")

# Use a consistent embedding model
embedding_model = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-Qwen2-1.5B-instruct")

# Function to load and chunk documents from CSV
def load_and_chunk_csv(csv_path):
    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)

    print(f"Loading CSV knowledge base from {csv_path}...")
    
    df = pd.read_csv(csv_path)
    
    if "Content" not in df.columns:
        raise ValueError("CSV file must contain a 'content' column with news articles.")

    print(f"Processing {len(df)} news articles...")
    
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Splitting content"):
        text = row["Content"]
        chunks = text_splitter.split_text(text)
        
        for chunk in chunks:
            all_documents.append(chunk)

    print(f"Total document chunks created: {len(all_documents)}")
    return all_documents

# Function to create FAISS vector store from CSV content
def create_vector_store(csv_path, vectorstore_path):
    print("Loading and chunking documents...")
    documents = load_and_chunk_csv(csv_path)
    
    if not documents:
        raise ValueError("No documents found in the knowledge base.")

    print("Creating FAISS vector store...")
    vectorstore = FAISS.from_texts(tqdm(documents, desc="Indexing documents"), embedding_model)
    vectorstore.save_local(vectorstore_path)
    
    print("Vector store created successfully.")
    return vectorstore

# Function to load or rebuild FAISS vector store
def load_or_create_vector_store(csv_path, vectorstore_path):
    if os.path.exists(vectorstore_path):
        print("FAISS index found. Deleting and recreating to avoid embedding mismatch...")
        os.system(f"rm -rf {vectorstore_path}")  # Remove old index
    return create_vector_store(csv_path, vectorstore_path)

# Function to load user queries
def load_queries(csv_path):
    try:
        print("Loading user queries...")
        df = pd.read_csv(csv_path)
        if "user" not in df.columns:
            raise ValueError("CSV file must contain a 'user' column.")
        print("User queries loaded successfully.")
        return df
    except Exception as e:
        print(f"Error loading queries: {e}")
        return None

# Function to retrieve context for queries
def retrieve_context(query, vectorstore, k=10, threshold=0.3):
    results = vectorstore.similarity_search_with_relevance_scores(query, k=k)
    filtered_results = [doc.page_content for doc, score in results if score >= threshold]
    
    if not filtered_results:
        return "No relevant context found."
    
    return "\n\n---\n\n".join(filtered_results)

# Function to retrieve and store context for all queries with tqdm
def retrieve_and_store_context(df, vectorstore, output_csv_path):
    tqdm.pandas(desc="Retrieving context")
    print("Retrieving context for queries...")
    
    df["context"] = df["user"].progress_apply(lambda query: retrieve_context(query, vectorstore))
    
    df.to_csv(output_csv_path, index=False, encoding="utf-8", quoting=csv.QUOTE_MINIMAL)
    print(f"✅ Context saved to {output_csv_path}")

# Execute the pipeline
print("Processing knowledge base (CSV-based)...")
vectorstore = load_or_create_vector_store(csv_knowledge_base_path, vectorstore_path)
df_queries = load_queries(csv_queries_path)

if df_queries is not None:
    retrieve_and_store_context(df_queries, vectorstore, output_csv_path)


Retrieving context:  43%|█████████████████████████████████████████▋                                                       | 1723/4006 [00:56<01:13, 31.12it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cc502a94-4b96-4f9f-a5ed-8d01d8842611', metadata={}, page_content='Quantum finance'), 0.1260111705543242), (Document(id='4b29a347-e0bc-4f56-82a4-b5335478d6ee', metadata={}, page_content='See also\n2007–2008 financial crisis\nOutline of finance'), 0.035198704767053224), (Document(id='d3d44fbc-79c9-4683-8ad5-6e39f4f35304', metadata={}, page_content="Agricultural sector depends significantly on external factors such as the weather, pest and disease outbreaks and prices of inputs and outputs, which are beyond the farmers' control. This translates to additional risks for agricultural financial institutions. These risks are exacerbated by the fact that they are common for most or all farmers in the area. For rural\nfinancial i

Retrieving context:  45%|███████████████████████████████████████████▊                                                     | 1811/4006 [00:58<01:10, 30.96it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cc502a94-4b96-4f9f-a5ed-8d01d8842611', metadata={}, page_content='Quantum finance'), 0.0784392639968049), (Document(id='2815a4a2-e2d6-46a6-9a31-e912c77cbf89', metadata={}, page_content='Purchasing insurance to ensure protection against unforeseen personal events;\nUnderstanding the effects of tax policies, subsidies, or penalties on the management of personal finances;\nUnderstanding the effects of credit on individual financial standing;\nDeveloping a savings plan or financing for large purchases (auto, education, home);\nPlanning a secure financial future in an environment of economic instability;\nPursuing a checking or a savings account;\nPreparing for retirement or other long term expenses.'), 0.059826878519606), (

Retrieving context:  51%|█████████████████████████████████████████████████▌                                               | 2047/4006 [01:06<01:04, 30.27it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='2815a4a2-e2d6-46a6-9a31-e912c77cbf89', metadata={}, page_content='Purchasing insurance to ensure protection against unforeseen personal events;\nUnderstanding the effects of tax policies, subsidies, or penalties on the management of personal finances;\nUnderstanding the effects of credit on individual financial standing;\nDeveloping a savings plan or financing for large purchases (auto, education, home);\nPlanning a secure financial future in an environment of economic instability;\nPursuing a checking or a savings account;\nPreparing for retirement or other long term expenses.'), 0.09914036299250673), (Document(id='85a1cdd5-494e-461f-8d47-199729cd13b7', metadata={}, page_content='Finance refers to monetary resources an

Retrieving context:  54%|████████████████████████████████████████████████████▍                                            | 2166/4006 [01:10<00:58, 31.35it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cc502a94-4b96-4f9f-a5ed-8d01d8842611', metadata={}, page_content='Quantum finance'), 0.13132335934063732), (Document(id='4b29a347-e0bc-4f56-82a4-b5335478d6ee', metadata={}, page_content='See also\n2007–2008 financial crisis\nOutline of finance'), 0.04676068113165266), (Document(id='2815a4a2-e2d6-46a6-9a31-e912c77cbf89', metadata={}, page_content='Purchasing insurance to ensure protection against unforeseen personal events;\nUnderstanding the effects of tax policies, subsidies, or penalties on the management of personal finances;\nUnderstanding the effects of credit on individual financial standing;\nDeveloping a savings plan or financing for large purchases (auto, education, home);\nPlanning a secure financial future in

Retrieving context:  62%|████████████████████████████████████████████████████████████▌                                    | 2502/4006 [01:21<00:48, 30.97it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cc502a94-4b96-4f9f-a5ed-8d01d8842611', metadata={}, page_content='Quantum finance'), 0.17686825536587358), (Document(id='4b29a347-e0bc-4f56-82a4-b5335478d6ee', metadata={}, page_content='See also\n2007–2008 financial crisis\nOutline of finance'), 0.15781063058100564), (Document(id='81493b73-d890-4f77-bf87-35a75cbd6875', metadata={}, page_content="Financial planning and forecasting\nCapital budgeting\nCapital structure\nWorking capital management\nRisk management\nFinancial analysis and reporting.\nThe discussion, however, extends to business strategy more broadly, emphasizing alignment with the company's overall strategic objectives; and similarly incorporates the managerial perspectives of planning, directing, and cont

...

Retrieving context:  78%|███████████████████████████████████████████████████████████████████████████▍                     | 3115/4006 [01:41<00:31, 28.71it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cc502a94-4b96-4f9f-a5ed-8d01d8842611', metadata={}, page_content='Quantum finance'), 0.08446162718052652), (Document(id='4b29a347-e0bc-4f56-82a4-b5335478d6ee', metadata={}, page_content='See also\n2007–2008 financial crisis\nOutline of finance'), 0.016077185359842705), (Document(id='7d9f406a-72e2-4ce9-9b3e-4e6de3e4a143', metadata={}, page_content='Areas of finance\nAs outlined, finance comprises, broadly, the three areas of personal finance, corporate finance, and public finance.These, in turn, overlap and employ various activities and sub-disciplines—chiefly investments, risk management, and quantitative finance.\n\nPersonal finance\nPersonal finance refers to the practice of budgeting to ensure enough funds are availa

Retrieving context:  90%|██████████████████████████████████████████████████████████████████████████████████████▉          | 3588/4006 [01:56<00:13, 31.00it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cc502a94-4b96-4f9f-a5ed-8d01d8842611', metadata={}, page_content='Quantum finance'), 0.20063022708783063), (Document(id='4b29a347-e0bc-4f56-82a4-b5335478d6ee', metadata={}, page_content='See also\n2007–2008 financial crisis\nOutline of finance'), 0.10712828660338236), (Document(id='81493b73-d890-4f77-bf87-35a75cbd6875', metadata={}, page_content="Financial planning and forecasting\nCapital budgeting\nCapital structure\nWorking capital management\nRisk management\nFinancial analysis and reporting.\nThe discussion, however, extends to business strategy more broadly, emphasizing alignment with the company's overall strategic objectives; and similarly incorporates the managerial perspectives of planning, directing, and cont

Retrieving context:  94%|███████████████████████████████████████████████████████████████████████████████████████████▍     | 3776/4006 [02:02<00:07, 31.88it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='4a0511db-343c-4775-869a-6edc87c08e8a', metadata={}, page_content='Behavioral finance\nBehavioral finance studies how the psychology of investors or managers affects financial decisions and markets \nand is relevant when making a decision that can impact either negatively or positively on one of their areas. With more in-depth research into behavioral finance, it is possible to bridge what actually happens in financial markets with analysis based on financial theory. Behavioral finance has grown over the last few decades to become an integral aspect of finance.\nBehavioral finance includes such topics as:\n\nEmpirical studies that demonstrate significant deviations from classical theories;\nModels of how psychology affec

Retrieving context:  98%|███████████████████████████████████████████████████████████████████████████████████████████████▍ | 3944/4006 [02:07<00:01, 31.63it/s]/tmp/ipykernel_27568/3928244748.py:87: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cc502a94-4b96-4f9f-a5ed-8d01d8842611', metadata={}, page_content='Quantum finance'), -0.10479904893873404), (Document(id='2815a4a2-e2d6-46a6-9a31-e912c77cbf89', metadata={}, page_content='Purchasing insurance to ensure protection against unforeseen personal events;\nUnderstanding the effects of tax policies, subsidies, or penalties on the management of personal finances;\nUnderstanding the effects of credit on individual financial standing;\nDeveloping a savings plan or financing for large purchases (auto, education, home);\nPlanning a secure financial future in an environment of economic instability;\nPursuing a checking or a savings account;\nPreparing for retirement or other long term expenses.'), -0.1106392534431917

✅ Context saved to ./queries_with_context_latest.csv


In [ ]:
# !pip install -U langchain-community

In [ ]:
# pip uninstall nltk 

In [ ]:
# !pip uninstall -y tensorflow
# !pip install --no-cache-dir tensorflow


In [2]:
# !pip uninstall -y safetensors
# !pip install --no-cache-dir safetensors


Found existing installation: safetensors 0.4.3
Uninstalling safetensors-0.4.3:
  Successfully uninstalled safetensors-0.4.3


In [5]:
# !pip uninstall -y transformers sentencepiece
# !rm -rf ~/.cache/huggingface


Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: sentencepiece 0.2.0
Uninstalling sentencepiece-0.2.0:
  Successfully uninstalled sentencepiece-0.2.0


In [6]:
pip install --no-cache-dir transformers sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 706.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 628.5 kB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llava 1.2.2.post1 requires httpx==0.24.0, but you have httpx 0.28.1 which is incompatible.
llava 1.2.2.post1 requires scikit-learn==1.2.2, but you have scikit-learn 1.3.2 which is incompatible.
llava 1.2.2.post1 requires sentencepiece==0.1.99, but you have sentencepiece 0.2.0 which is incompatible.
llava 1.2.2.post1 requires tokenizers==0.15.1, but you have tokenizers 0.20.3 which is incompatible.
llava 1.2.2.post1 requires torch==2.1.2, but you have torch 2.4.1 which is incompatible.
llava 1.2.2.post1 requires torchvision==0.16.2, but you have torchvision 0.19.1 which is incompatible.
llava 1.2.2.post1 requires transformers==4.37.2, but you 

In [3]:
!pip uninstall -y sentencepiece transformers
!pip install --no-cache-dir sentencepiece transformers


Found existing installation: sentencepiece 0.1.99
Uninstalling sentencepiece-0.1.99:
  Successfully uninstalled sentencepiece-0.1.99
Found existing installation: transformers 4.37.2
Uninstalling transformers-4.37.2:
  Successfully uninstalled transformers-4.37.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 101.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llava 1.2.2.post1 requires httpx==0.24.0, but you have httpx 0.28.1 which is incompatible.
llava 1.2.2.post1 requires scikit-learn==1.2.2, but you h

In [17]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [15]:
!pip install --force-reinstall torch torchvision torchaudio


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.3 MB/s eta 0:00:0000:0100:08
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:0000:0100:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.5 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━

    Uninstalling mpmath-1.3.0:
      Successfully uninstalled mpmath-1.3.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: pillow
    Found existing installation: pillow 10.3.0
    Uninstalling pillow-10.3.0:
      Successfully uninstalled pillow-10.3.0
  Attempting uninstall: nvidia-nvtx-cu12
    Found existing installation: nvidia-nvtx-cu12 12.1.105
    Uninstalling nvidia-nvtx-cu12-12.1.105:
      Successfully uninstalled nvidia-nvtx-cu12-12.1.105
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.40
    Uninstalling nvidia-nvjitlink-cu12-12.5.40:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.5.40
  Attem

In [12]:
!pip install --upgrade --force-reinstall pydantic


  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.3 MB/s eta 0:00:00ta 0:00:01
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: annotated-types
    Found existing installation: annotated-types 0.7.0
    Uninstalling annotated-types-0.7.0:
      Successfully uninstalled annotated-types-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
ERROR: pip's dependency resolver does not curr

In [10]:
!pip install --upgrade --force-reinstall pandas



  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 10.0 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2024.1
    Uninstalling tzdata-2024.1:
      Successfully uninstalled tzdata-2024.1
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0

In [21]:
!find ~/.miniconda3/envs/finbot3/lib/python3.8/site-packages/ -name "*.so" -delete
!find ~/.miniconda3/envs/finbot3/lib/python3.8/site-packages/ -name "*.pyc" -delete


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
pip install --upgrade --force-reinstall scikit-learn numpy


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 15.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 2.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.5.0
    Uninstalling threadpoolctl-3.5.0:
      Successfully uninstalled threadpoolctl-3.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstall

In [5]:
pip install unstructured

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install openai

Note: you may need to restart the kernel to use updated packages.
